In [22]:
import sqlite3
import pandas as pd
import numpy as np
import time
import math
import pickle
import multiprocessing as mp
import gc

from datetime import datetime, timedelta

! mkdir -p out/frames

In [12]:
def read_data():
    with sqlite3.connect("data.sqlite3") as db:
        return pd.read_sql_query(f"""
            -- Dedupe rows in case scrape was executed multiple times
            with max_scrape as (
                select id, max(scrapeTime) as scrapeTime
                from earthquakes
                group by id
            )
            select time, latitude, longitude, depth, mag
            from earthquakes
            inner join max_scrape using (id, scrapeTime)
            where status <> 'deleted'
        """, db, parse_dates=None)

df = read_data()
df['datetime'] = pd.to_datetime(df.time)
df['unix'] = (df.datetime.astype(np.int64) / 1e9).astype(np.int64)
df['energy'] = 10 ** (1.5 * (df.mag + 3.2))

In [13]:
# Targeting 5min@60fps
target_fps = 60
target_len_minutes = 5

# Time range to draw
start_time = datetime(2010, 1, 1, 0, 0).timestamp()
end_time = datetime(2019, 7, 8, 0, 0).timestamp()

# Number of frames for quakes to fade away
fadeaway_frames = 60

target_frame_count = target_fps * target_len_minutes * 60
total_data_unix = end_time - start_time
unix_per_frame = int(total_data_unix / target_frame_count)


In [17]:
min_depth = 1
max_depth = 800
 
def draw_data(end_unix):
    import os
    # Bypass annoying JupyterLab issue with Conda
    os.environ.setdefault('PROJ_LIB', '/home/randombk/.conda/envs/data-analytics/share/proj')
    
    # Avoid matplotlib multiprocessing errors by importing inside the function
    import matplotlib.pyplot as plt
    from mpl_toolkits.basemap import Basemap
    from pandas.plotting import register_matplotlib_converters
    register_matplotlib_converters()
    
    frame_start_time = time.time()
    
    # Prepare main map data
    data = df[
        (df.unix < end_unix) & 
        (df.unix > end_unix - unix_per_frame * fadeaway_frames)
    ].copy()
    data['opacity'] = 1 - ((end_unix - data.unix) / unix_per_frame) / fadeaway_frames
    
    # Prepare bottom heatmap data
    heatmap_data = df[(df.unix < end_unix) & df.depth.notnull()]
    
    # Prepare plot
    fig = plt.figure(figsize=(19,12))
    ax_map = fig.add_axes([0.05, 0.2, 0.9, 0.8], xticklabels=[], yticklabels=[], ylim=(-1.2, 1.2))
    ax_depth_heatmap = fig.add_axes(
        [0.05, 0.05, 0.9, 0.15],
        xlim=(datetime.utcfromtimestamp(start_time), datetime.utcfromtimestamp(end_time)),
        ylim=(-max_depth, min_depth),
        ylabel="Epicenter Depth (km)"
    )

    title = [
        "Global Earthquakes Greater Than 2.5 Magnitude",
        datetime.utcfromtimestamp(end_unix).strftime('%Y-%m-%d %H:%M:%S UTC'),
    ]
    ax_map.set_title("\n".join(title))

    # Draw background map
    basemap = Basemap(projection='robin',lon_0=0, ax=ax_map)
    basemap.drawcoastlines()
    basemap.drawparallels(np.arange(-90, 90, 30),labels=[1, 0, 0, 0])
    basemap.drawmeridians(np.arange(basemap.lonmin,basemap.lonmax+30,60),labels=[0,0,0,1])
    
    # Draw earthquakes
    quake_x, quake_y = basemap(data.longitude.to_list(), data.latitude.to_list())
    quake_colors = np.zeros((len(data), 4))
    quake_colors[:,0] = np.log(np.clip(data.depth, min_depth, max_depth)) / np.log(max_depth)
    quake_colors[:,2] = 1.0 - np.log(np.clip(data.depth, min_depth, max_depth)) / np.log(max_depth)
    quake_colors[:,3] = data.opacity
    quake_sizes = (data.energy/1.0e7)**0.5 / 10
    ax_map.scatter(quake_x, quake_y, s=quake_sizes, color=quake_colors)
    
    # Draw heatmap
    heatmap_colors = np.zeros((len(heatmap_data), 4))
    heatmap_colors[:,0] = np.log(np.clip(heatmap_data.depth, min_depth, max_depth)) / np.log(max_depth)
    heatmap_colors[:,2] = 1.0 - np.log(np.clip(heatmap_data.depth, min_depth, max_depth)) / np.log(max_depth)
    heatmap_colors[:,3] = 0.3
    heatmap_sizes = (heatmap_data.energy/1.0e7)**0.5 / 100
    ax_depth_heatmap.scatter(
        heatmap_data.datetime, 
        -np.clip(heatmap_data.depth, min_depth, max_depth), 
        s=heatmap_sizes, 
        color=heatmap_colors
    )


    # Draw Metrics
    metrics = [
        f"Total Events: {len(heatmap_data)}",
        f"Mag 7+: {len(heatmap_data[heatmap_data.mag >= 7])}",
        f"Mag 6-6.9: {len(heatmap_data[(heatmap_data.mag >= 6) & (heatmap_data.mag < 7)])}",
        f"Mag 5-5.9: {len(heatmap_data[(heatmap_data.mag >= 5) & (heatmap_data.mag < 6)])}",
        f"Mag <5: {len(heatmap_data[heatmap_data.mag < 5])}",
    ]
    for idx, val in enumerate(metrics):
        ax_map.text(
            -600000, 17000000 - 320000 * idx, 
            val,
            fontsize='large',
        )

    # Draw Magnitude Label
    ax_map.text(
        -600000, 500000, 
        "Magnitude",
        rotation=90,
        fontsize='x-large'
    )
    legend_mags = np.array([8, 7, 6, 5, 4])
    legend_xpos = [1300000, 3200000, 4200000, 4800000, 5200000]
    legend_ypos = [1500000, 850000, 620000, 500000, 470000]
    legend_energies = 10 ** (1.5 * (legend_mags + 3.2))
    legend_mag_sizes = (legend_energies/1.0e7)**0.5 / 10
    for i in range(len(legend_mags)):
        ax_map.scatter(
            legend_xpos[i],
            legend_ypos[i],
            s=legend_mag_sizes[i],
            c = [(0.8,0.8,0.8)],
            edgecolor = [(0.3,0.3,0.3)]
        )
        ax_map.annotate(str(legend_mags[i]), (legend_xpos[i], 0), ha='center')
    
    # Draw footnotes
    footnotes = [
        'Created by David Li',
        '/u/dlp_randombk',
        '',
        'Source: U.S. Geological Survey',
        'Accessed 2019-07-07',
    ]
    for idx, val in enumerate(footnotes):
        ax_map.text(
            33500000, 1320000 - 320000 * idx, 
            val,
            fontsize='large',
            horizontalalignment='right'
        )

    fig.savefig(f"out/frames/{int(end_unix)}.png", dpi=160, transparent=False)
    
    fig.clf()
    plt.close(fig)
    plt.close()
    del data
    del heatmap_data
    gc.collect()
    
    return (end_unix, time.time() - frame_start_time)


In [21]:
frame_times = []

if __name__ == '__main__':
    # Prepare list of jobs
    jobs = range(int(start_time), int(end_time + unix_per_frame), unix_per_frame)
    print(f"Using {mp.cpu_count()} cores...")

    # There's a memory leak somewhere. We'll just hack our way around it.
    with mp.Pool(processes=mp.cpu_count(), maxtasksperchild=4) as pool:
        frame_times = pool.map(draw_data, jobs, chunksize=4)
        with open('out/frame_times.pickle', 'wb') as fp:
            pickle.dump(frame_times, fp)

Using 24 cores...
